### Import the necessary libraries if you are starting a new notebook. Using the same data as the previous lab: we_fn_use_c_marketing_customer_value_analysis.csv

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import re

In [4]:
customer_df = pd.read_csv('..\\lab-4.02\\files_for_lab\\we_fn_use_c_marketing_customer_value_analysis.csv')
customer_df.columns = [i.lower().replace(' ', '_') for i in customer_df.columns]
customer_df.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [5]:
customer_df_num = customer_df.select_dtypes(np.number)

In [6]:
def continuous(df):

    discrete_list = []
    continuous_list = []
    
    for col in df:
        
        if (df[col].nunique() < (customer_df_num['income'].nunique()/10)) & (df[col].dtypes == 'int64'):
            discrete_list.append(col)
        
        else:
            continuous_list.append(col)
    
    
    discrete_df = pd.DataFrame(customer_df_num[discrete_list])
    continuous_df = pd.DataFrame(customer_df_num[continuous_list]) 
                    

    return discrete_df, continuous_df

In [7]:
discrete_customer, continuous_customer = continuous(customer_df_num)

### Find all of the categorical data. Save it in a categorical_df variable.


In [8]:
categorical_df = customer_df.select_dtypes(object).copy()
categorical_df.head()

,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize



### Check for NaN values.


In [9]:
categorical_df.isna().sum()

customer             0
state                0
response             0
coverage             0
education            0
effective_to_date    0
employmentstatus     0
gender               0
location_code        0
marital_status       0
policy_type          0
policy               0
renew_offer_type     0
sales_channel        0
vehicle_class        0
vehicle_size         0
dtype: int64


### Check all unique values of columns.


In [10]:
def uniques(df):
    for col in df:
        print(df[col].unique())

In [11]:
uniques(categorical_df)

['BU79786' 'QZ44356' 'AI49188' ... 'TD14365' 'UP19263' 'Y167826']
['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
['No' 'Yes']
['Basic' 'Extended' 'Premium']
['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
['2/24/11' '1/31/11' '2/19/11' '1/20/11' '2/3/11' '1/25/11' '1/18/11'
 '1/26/11' '2/17/11' '2/21/11' '1/6/11' '2/6/11' '1/10/11' '1/17/11'
 '1/5/11' '2/27/11' '1/14/11' '1/21/11' '2/5/11' '1/29/11' '2/28/11'
 '2/12/11' '2/2/11' '2/7/11' '1/22/11' '2/13/11' '1/15/11' '1/8/11'
 '1/11/11' '1/28/11' '2/8/11' '2/23/11' '1/2/11' '2/16/11' '1/27/11'
 '1/23/11' '1/9/11' '2/11/11' '2/4/11' '2/1/11' '2/15/11' '2/26/11'
 '1/16/11' '1/1/11' '2/10/11' '1/24/11' '2/25/11' '1/12/11' '2/9/11'
 '1/19/11' '1/4/11' '2/14/11' '2/20/11' '2/18/11' '1/3/11' '1/13/11'
 '1/30/11' '2/22/11' '1/7/11']
['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']
['F' 'M']
['Suburban' 'Rural' 'Urban']
['Married' 'Single' 'Divorced']
['Corporate Auto' 'Personal Auto' 'Special Auto']
['Co


### Check dtypes. Do they all make sense as categorical data?


In [12]:
categorical_df.dtypes

customer             object
state                object
response             object
coverage             object
education            object
effective_to_date    object
employmentstatus     object
gender               object
location_code        object
marital_status       object
policy_type          object
policy               object
renew_offer_type     object
sales_channel        object
vehicle_class        object
vehicle_size         object
dtype: object

Effective to date does not make sense as an object, it should be changed to a datetime format. The customer column is technically categorical but it doesn't give us any usefull information so we think we should drop it. 

In [13]:
categorical_df['effective_to_date'] = pd.to_datetime(customer_df['effective_to_date'])


### Does any column contain alpha and numeric data? Decide how to clean it.


We are dropping the customer column, and then we are removing the alpha part from policy and offer type. 

In [14]:
categorical_df.drop(['customer'], axis = 1, inplace = True)

In [15]:
def alphac(x):
    if 'Corporate L' in x:
        return x.replace('Corporate L', '')
    elif 'Personal L' in x:
        return x.replace('Personal L', '')
    elif 'Special L' in x:
        return x.replace('Special L', '')
    else:
        return x

In [16]:
categorical_df['policy'] = categorical_df['policy'].apply(alphac)
categorical_df['policy'].unique()
# We remove the alphabetical part because in the variable 'policy_type' we already see if it is Personal, Special or Corporate. 

array(['3', '2', '1'], dtype=object)

In [17]:
def offer(x):
    if 'Offer1' in x:
        return '1'
    elif 'Offer2' in x:
        return '2'
    elif 'Offer3' in x:
        return '3'
    else:
        return '4'

In [18]:
categorical_df['renew_offer_type'] = categorical_df['renew_offer_type'].apply(offer)
categorical_df['renew_offer_type'].unique()

array(['1', '3', '2', '4'], dtype=object)


### Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions.


In [19]:
categorical_df.head()

,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,No,Basic,Bachelor,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,3,1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Bachelor,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,3,3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Bachelor,2011-02-19,Employed,F,Suburban,Married,Personal Auto,3,1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Bachelor,2011-01-20,Unemployed,M,Suburban,Married,Corporate Auto,2,1,Call Center,SUV,Medsize
4,Washington,No,Basic,Bachelor,2011-02-03,Employed,M,Rural,Single,Personal Auto,1,1,Agent,Four-Door Car,Medsize


We think the data is now clean and it is not necessary to do anything else. 


### Compare policy_type and policy. What information is contained in these columns. Can you identify what is important?


In [20]:
categorical_df.rename(columns = {'policy': 'policy_level'}, inplace = True)

In [21]:
categorical_df[['policy_type', 'policy_level']]

,policy_type,policy_level
0,Corporate Auto,3
1,Personal Auto,3
2,Personal Auto,3
3,Corporate Auto,2
4,Personal Auto,1
...,...,...
9129,Personal Auto,1
9130,Corporate Auto,3
9131,Corporate Auto,2
9132,Personal Auto,2


We previously edited the policy column so it only showed the level, because we already have the type (Personal, Corporate or Special) in the other column. We think that now it will be easier to encode. 


### Check number of unique values in each column, can they be combined in any way to ease encoding? Comment your thoughts and make those changes.

In [22]:
uniques(categorical_df)

['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
['No' 'Yes']
['Basic' 'Extended' 'Premium']
['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
['2011-02-24T00:00:00.000000000' '2011-01-31T00:00:00.000000000'
 '2011-02-19T00:00:00.000000000' '2011-01-20T00:00:00.000000000'
 '2011-02-03T00:00:00.000000000' '2011-01-25T00:00:00.000000000'
 '2011-01-18T00:00:00.000000000' '2011-01-26T00:00:00.000000000'
 '2011-02-17T00:00:00.000000000' '2011-02-21T00:00:00.000000000'
 '2011-01-06T00:00:00.000000000' '2011-02-06T00:00:00.000000000'
 '2011-01-10T00:00:00.000000000' '2011-01-17T00:00:00.000000000'
 '2011-01-05T00:00:00.000000000' '2011-02-27T00:00:00.000000000'
 '2011-01-14T00:00:00.000000000' '2011-01-21T00:00:00.000000000'
 '2011-02-05T00:00:00.000000000' '2011-01-29T00:00:00.000000000'
 '2011-02-28T00:00:00.000000000' '2011-02-12T00:00:00.000000000'
 '2011-02-02T00:00:00.000000000' '2011-02-07T00:00:00.000000000'
 '2011-01-22T00:00:00.000000000' '2011-02-13T00:00:00.00

In [23]:
def counts(df):
    for col in df:
        print(df[col].value_counts())

counts(categorical_df)

California    3150
Oregon        2601
Arizona       1703
Nevada         882
Washington     798
Name: state, dtype: int64
No     7826
Yes    1308
Name: response, dtype: int64
Basic       5568
Extended    2742
Premium      824
Name: coverage, dtype: int64
Bachelor                2748
College                 2681
High School or Below    2622
Master                   741
Doctor                   342
Name: education, dtype: int64
2011-01-10    195
2011-01-27    194
2011-02-14    186
2011-01-26    181
2011-01-17    180
2011-01-19    179
2011-01-31    178
2011-01-03    178
2011-01-20    173
2011-02-26    169
2011-01-28    169
2011-02-19    168
2011-01-05    167
2011-02-27    167
2011-01-11    166
2011-02-04    164
2011-02-10    161
2011-02-28    161
2011-01-02    160
2011-01-21    160
2011-01-29    160
2011-02-22    158
2011-02-05    158
2011-02-03    158
2011-02-07    157
2011-02-12    156
2011-01-23    155
2011-02-01    154
2011-01-18    154
2011-01-15    153
2011-01-14    152
2011-02-11   

In [24]:
categorical_df['vehicle_class'] = np.where(categorical_df['vehicle_class'] == 'Luxury SUV', 'Luxury Car', categorical_df['vehicle_class'])
categorical_df['vehicle_class'] = categorical_df['vehicle_class'].str.replace('Luxury Car','Luxury')
categorical_df['vehicle_class'].value_counts()

Four-Door Car    4621
Two-Door Car     1886
SUV              1796
Sports Car        484
Luxury            347
Name: vehicle_class, dtype: int64

In [25]:
categorical_df.drop(['vehicle_size'], axis = 1, inplace = True)

Since the value count for Luxury Car and Luxury SUV is so low compared with the others, I have decided to merge them. I don't think the fact that the car is an SUV or not matters in the total claim amount, but the car being a luxury brand surely does. 

# Today's Lab

### We will start with removing outliers, if you have not already done so. We have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

In [26]:
discrete_customer_n = discrete_customer['number_of_open_complaints']
discrete_customer_d = discrete_customer.drop(['number_of_open_complaints'], axis = 1)

In [27]:
def outliers(df):

    for column in df:
        upper = np.percentile(df[column],75)
        lower = np.percentile(df[column],25)
        iqr = upper - lower
        upper_limit = upper + 1.5 * iqr
        lower_limit = lower - 1.5 * iqr
        cleaned_df = df[(df[column]>lower_limit) & (df[column]<upper_limit)]
    return cleaned_df

In [28]:
discrete_customer_d = outliers(discrete_customer)
continuous_customer = outliers(continuous_customer)

### The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

In [29]:
customer_df['day'] = pd.to_datetime(customer_df['effective_to_date']).dt.day
customer_df['month'] = pd.to_datetime(customer_df['effective_to_date']).dt.month
customer_df['year'] = pd.to_datetime(customer_df['effective_to_date']).dt.year
date = customer_df[['day', 'month', 'year']]

In [30]:
total = pd.concat([categorical_df, discrete_customer_d, discrete_customer_n, continuous_customer, date], axis = 1)
total.dropna(axis = 0, inplace = True)
total.shape

(8283, 26)

In [31]:
total['coverage'] = total['coverage'].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})

### Create a copy of the dataframe for the data wrangling.

In [32]:
total_data = total.copy()

### In this final lab, we will model our data. Import sklearn train_test_split and separate the data.

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
y = total_data['total_claim_amount']
X = total_data.drop(['total_claim_amount'], axis = 1)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

numericals_train = X_train.select_dtypes(np.number)
numericals_test = X_test.select_dtypes(np.number)

categoricals_train= X_train.select_dtypes(object)
categoricals_test= X_test.select_dtypes(object)

### Normalize the continuous variables. You can use any one method you want.

In [36]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(numericals_train)
numericals_train_standardized = transformer.transform(numericals_train)
numericals_test_standardized = transformer.transform(numericals_test)

### Encode the categorical variables (See the hint below for encoding categorical data!!!)

In [37]:
categoricals_train['employmentstatus'].value_counts()

Employed         3975
Unemployed       1496
Medical Leave     284
Disabled          266
Retired           191
Name: employmentstatus, dtype: int64

In [38]:
categoricals_train['location_code'].value_counts()

Suburban    3794
Rural       1274
Urban       1144
Name: location_code, dtype: int64

In [39]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(categoricals_train)
categoricals_train_encoded = encoder.transform(categoricals_train).toarray()
categoricals_test_encoded = encoder.transform(categoricals_test).toarray()

In [40]:
X_train = np.concatenate((numericals_train_standardized, categoricals_train_encoded),axis=1)
X_test = np.concatenate((numericals_test_standardized, categoricals_test_encoded),axis=1)

### Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

In [41]:
X_train.dtype

dtype('float64')

In [42]:
X_test.dtype

dtype('float64')

### Try a simple linear regression with all the data to see whether we are getting good results.

In [43]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression().fit(X_train,y_train)

score = LR.score(X_test, y_test)

y_pred = LR.predict(X_test)

score

0.7937939291836547

### Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

In [44]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

In [45]:
def model(X_train, X_test, y_train, y_test):
    
    LR = LinearRegression()
    KNN = KNeighborsRegressor(n_neighbors=6)
    MLP = MLPRegressor()
    
    model_list = [LR, KNN, MLP]
    
    for i in model_list:
        
        i.fit(X_train,y_train)
        score = i.score(X_test, y_test)
        print(i, 'model:', score)

### Use the function to check `LinearRegressor` and `KNeighborsRegressor`.

### You can check also the MLPRegressor for this task!

In [52]:
model(X_train, X_test, y_train, y_test)

LinearRegression() model: 0.7937939291836547
KNeighborsRegressor(n_neighbors=6) model: 0.5818492909967742
MLPRegressor() model: 0.8092008434224144


C:\Users\herat\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Check and discuss the results.

The k-neighbors model doesn't work as well as the linear regression in this case, but the MLP is a little bit better. 